Importo librerias de Amazon

In [1]:
import io
import boto3
import os
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role

Importo librerias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from sklearn import preprocessing
#from sklearn.neighbors import KNeighborsClassifier

Cargo CSVs

In [3]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('bucketcursoitba')
s3 = boto3.client('s3')
clients = []
workshops = []
results = []
results_bad = []

names = ['customers','vehicles']
names_workshop = ['workshops']

for i in range(len(names)):
    obj = s3.get_object(Bucket = 'bucketcursoitba', Key = f'raw/table_{names[i]}.csv')
    clients.append(pd.read_csv(obj['Body']))

obj = s3.get_object(Bucket = 'bucketcursoitba', Key = 'raw/table_workshops.csv')
workshops.append(pd.read_csv(obj['Body']))

obj = s3.get_object(Bucket = 'bucketcursoitba', Key = 'raw/table_results.csv')
results.append(pd.read_csv(obj['Body']))

In [38]:
results_bad = []
obj = s3.get_object(Bucket = 'bucketcursoitba', Key = 'raw/table_results23.csv')
results_bad.append(pd.read_csv(obj['Body']))

Primera limpieza sacos valores nulos y duplicados

In [4]:
clients[0] = clients[0].dropna(subset = ["dnb_prim_lon"])
clients[0] = clients[0].dropna(subset = ["dnb_prim_lat"])
clients[0] = clients[0].dropna(subset = ["dnb_sic_major"])
clients[0] = clients[0].drop_duplicates(subset = ["customer_guid"], keep='first')
clients[1] = clients[1].dropna(subset = ["vin"])
clients[1] = clients[1].drop_duplicates(subset = ["vin"], keep=False)
workshops[0] = workshops[0].dropna(subset = ["workshopcapacity"])

Uno tablas de vehiculos y clientes

In [46]:
Global = pd.merge(left=clients[0],right=clients[1],left_on='customer_guid',right_on='customer_guid', validate='one_to_many')
#Global = pd.DataFrame(clients[0])
#Global_workshops = pd.merge(left=workshops[0],right=workshops[1],left_on='dealerid',right_on='dealerid',validate='many_to_one')
Global_workshops = pd.DataFrame(workshops[0])
Global_results = pd.DataFrame(results[0])
Global_results_bad = pd.DataFrame(results_bad[0])

Primero busco y renomobro las columnas _T_ = Camiones, _B_ = Colectivos
si un cliente tiene los 2 es una _A_... luego borro los duplicados

In [6]:
Global.loc[Global['product'].str.contains('_T_'), 'product'] = 'T'
Global.loc[Global['product'].str.contains('_B_'), 'product'] = 'B'
Global=Global.drop_duplicates(subset = ['customer_guid', 'product'], keep = 'last')

Global['product'].mask(Global['customer_guid'].duplicated(keep=False), 'A', inplace=True)
Global=Global.drop_duplicates(subset = ['customer_guid', 'product'], keep = 'last')

In [7]:
conditions = [
    (Global['product'] == 'B'),
    (Global['product'] == 'T'),
    ]

# create a list of the values we want to assign for each condition
values = [1, 0]

# create a new column and use np.select to assign values to it using our lists as arguments
Global['bus'] = np.select(conditions, values)

Borro los Dealers que no tienen talleres
Para facilitar el modelo y dado que solo hay un taller que solo ve colectivos y esta muy cercano a uno que ve ambos voy a eliminar dicho taller del dataframe

In [8]:
Global_workshops1 = Global_workshops[ (Global_workshops['servicebus'] == 0) & (Global_workshops['servicetruck'] <= 0) ].index
Global_workshops.drop(Global_workshops1 , inplace=True)
Global_workshops.drop(Global_workshops[Global_workshops['dealerid'] == 40925].index, inplace = True)

Creo los datasets para entrenamiento

In [9]:
train_features = Global_workshops[["longitude","latitude","servicebus"]]
train_labels = Global_workshops["dealerid"]

train_features_np= train_features.to_numpy()
train_labels_np= train_labels.to_numpy()

Subo datos para entrenamiento a bucket

In [10]:
import io
import boto3
import os
import sagemaker
import sagemaker.amazon.common as smac

bucket = "bucketcursoitba"
prefix = "knn"
key = "processed"

print('train_features shape = ', train_features.shape)
print('train_labels shape = ', train_labels.shape)

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, train_features_np, train_labels_np)
buf.seek(0)

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

train_features shape =  (85, 3)
train_labels shape =  (85,)
uploaded training data location: s3://bucketcursoitba/knn/train/processed


Genero funcion para entrenamiento

In [56]:
import matplotlib.pyplot as plt

import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer


def trained_estimator_from_hyperparams(s3_train_data, hyperparams, output_path, s3_test_data=None):
    """
    Create an Estimator from the given hyperparams, fit to training data, 
    and return a deployed predictor

    """
    # set up the estimator
    knn = sagemaker.estimator.Estimator(get_image_uri(boto3.Session().region_name, "knn"),
        get_execution_role(),
        train_instance_count=1,
        serializer=CSVSerializer(),
        deserializer=JSONDeserializer(),
        train_instance_type='ml.m5.xlarge',
        output_path=output_path,
        sagemaker_session=sagemaker.Session())
    knn.set_hyperparameters(**hyperparams)

    # train a model. fit_input contains the locations of the train and test data
    fit_input = {'train': s3_train_data}
    if s3_test_data is not None:
        fit_input['test'] = s3_test_data
    knn.fit(fit_input)
    return knn

Configuro Hiperparametros

In [12]:
hyperparams = {
    'feature_dim': 3,
    'k': 1,
    'sample_size': 200000,
    'predictor_type': 'classifier' 
}
output_path = 's3://' + bucket + '/' + prefix + '/output'


Entreno el modelo

In [57]:
knn_estimator = trained_estimator_from_hyperparams(s3_train_data, hyperparams, output_path)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-12-11 00:00:50 Starting - Starting the training job...
2022-12-11 00:01:10 Starting - Preparing the instances for trainingProfilerReport-1670716850: InProgress
......
2022-12-11 00:02:06 Downloading - Downloading input data...
2022-12-11 00:02:46 Training - Downloading the training image...............
2022-12-11 00:05:07 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[12/11/2022 00:05:17 INFO 139800298325824] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-conf.json: {'_kvstore': 'dist_async', '_log_level': 'info', '_num_gpus': 'auto', '_num_kv_servers': '1', '_tuning_objective_metric': '', '_faiss_index_nprobe': '5', 'epochs': '1', 'feature_dim': 'auto', 'faiss_index_ivf_nlists': 'auto', 'index_metric': 'L2', 'index_type': 'faiss.Flat', 'mini_batch_size': '5000', '_enable_profiler': 'false'}
[12/11/2022 00:0

Configuro Endpoint

In [14]:
def predictor_from_estimator(knn_estimator, estimator_name, instance_type, endpoint_name=None): 
    knn_predictor = knn_estimator.deploy(initial_instance_count=1, instance_type=instance_type,endpoint_name=endpoint_name)
   # knn_predictor.content_type = 'text/csv'
    knn_predictor.serializer = csv_serializer
    knn_predictor.deserializer = json_deserializer
    return knn_predictor

Creo Endpoint

In [15]:
import time

instance_type = 'ml.m5.xlarge'
model_name = 'knn_%s'% instance_type
endpoint_name = 'emiliano-%s'% (str(time.time()).replace('.','-'))
print('setting up the endpoint..')
predictor = predictor_from_estimator(knn_estimator, model_name, instance_type, endpoint_name)

setting up the endpoint..
----------!

----------------------------------------------------------------------------------------------------

Configuro parametros de test

In [47]:
train_labels_np= train_labels.to_numpy()

In [48]:
datosClientes = Global[["dnb_prim_lon","dnb_prim_lat","bus"]]
datosClientes.columns = ['longitude', 'latitude',"servicebus"]
datosClientesBus = datosClientes[datosClientes['servicebus'] == 1]

KeyError: "['bus'] not in index"

Datos con error

In [49]:
test_features = datosClientesBus[["longitude","latitude","servicebus"]]
test_features_np= test_features.to_numpy()

test_labels = Global_results_bad["dealerid"]
#Global_results_bad.columns = ["dealerid"]
test_labels_np= test_labels.to_numpy()

test_labels

0      40457
1      40846
2      40846
3      40784
4      40487
       ...  
171    40286
172    40846
173    40585
174     1111
175    40067
Name: dealerid, Length: 176, dtype: int64

In [50]:
bucket = "bucketcursoitba"
prefix = "knn"
key = "processed"

print(f"test_features shape = {test_features.shape}")
print(f"test_labels shape = {test_labels.shape}")

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, test_features_np, test_labels_np)
buf.seek(0)

boto3.resource("s3").Bucket(bucket).Object(os.path.join(prefix, "test", key)).upload_fileobj(buf)
s3_test_data = f"s3://{bucket}/{prefix}/test/{key}"
print(f"uploaded test data location: {s3_test_data}")

test_features shape = (176, 3)
test_labels shape = (176,)
uploaded test data location: s3://bucketcursoitba/knn/test/processed


In [51]:
import time
batches = np.array_split(test_features_np, 100)
print(f"data split into 100 batches, of size {batches[0].shape[0]}.")

# obtain an np array with the predictions for the entire test set
start_time = time.time()
predictions = []
for batch in batches:
    result = predictor.predict(batch, initial_args={"ContentType": "text/csv"})
    cur_predictions = np.array(
        [result["predictions"][i]["predicted_label"] for i in range(len(result["predictions"]))]
    )
    predictions.append(cur_predictions)
predictions = np.concatenate(predictions)
run_time = time.time() - start_time

test_size = test_labels.shape[0]
num_correct = sum(predictions == test_labels_np)
accuracy = num_correct / float(test_size)
print("time required for predicting %d data point: %.2f seconds" % (test_size, run_time))
print("accuracy of model: %.1f%%" % (accuracy * 100))
predictions

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_seri

data split into 100 batches, of size 2.


The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_seri

time required for predicting 176 data point: 1.19 seconds
accuracy of model: 92.6%


array([40457., 40846., 40846., 40784., 40487., 40620., 40267., 40650.,
       40015., 40457., 40484., 40360., 40784., 40650., 40015., 40340.,
       40457., 40595., 40230., 40210., 40210., 40487., 40487., 40360.,
       40457., 40410., 40287., 40340., 40623., 40680., 40846., 40288.,
       40680., 40483., 40595., 40599., 40067., 40770., 40435., 40015.,
       40466., 40487., 40267., 40814., 40410., 40846., 40360., 40015.,
       40360., 40457., 40210., 40210., 40650., 40585., 40287., 40287.,
       40210., 40680., 40457., 40067., 40487., 40286., 40287., 40554.,
       40846., 40266., 40650., 40487., 40680., 40410., 40784., 40360.,
       40210., 40680., 40410., 40680., 40435., 40457., 40680., 40487.,
       40210., 40846., 40457., 40585., 40487., 40210., 40585., 40457.,
       40210., 40680., 40595., 40585., 40554., 40770., 40340., 40680.,
       40680., 40487., 40487., 40015., 40680., 40286., 40585., 40554.,
       40340., 40340., 40210., 40680., 40680., 40435., 40015., 40585.,
      

Datos 100%

In [52]:
test_features = datosClientesBus[["longitude","latitude","servicebus"]]
test_features_np= test_features.to_numpy()

test_labels = Global_results["dealerid"]
test_labels_np= test_labels.to_numpy()

test_labels

0      40457
1      40846
2      40846
3      40784
4      40487
       ...  
171    40286
172    40846
173    40585
174    40210
175    40067
Name: dealerid, Length: 176, dtype: int64

In [53]:
bucket = "bucketcursoitba"
prefix = "knn"
key = "processed"

print(f"test_features shape = {test_features.shape}")
print(f"test_labels shape = {test_labels.shape}")

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, test_features_np, test_labels_np)
buf.seek(0)

boto3.resource("s3").Bucket(bucket).Object(os.path.join(prefix, "test", key)).upload_fileobj(buf)
s3_test_data = f"s3://{bucket}/{prefix}/test/{key}"
print(f"uploaded test data location: {s3_test_data}")

test_features shape = (176, 3)
test_labels shape = (176,)
uploaded test data location: s3://bucketcursoitba/knn/test/processed


In [54]:
batches = np.array_split(test_features_np, 100)
print(f"data split into 100 batches, of size {batches[0].shape[0]}.")

# obtain an np array with the predictions for the entire test set
start_time = time.time()
predictions = []
for batch in batches:
    result = predictor.predict(batch, initial_args={"ContentType": "text/csv"})
    cur_predictions = np.array(
        [result["predictions"][i]["predicted_label"] for i in range(len(result["predictions"]))]
    )
    predictions.append(cur_predictions)
predictions = np.concatenate(predictions)
run_time = time.time() - start_time

test_size = test_labels.shape[0]
num_correct = sum(predictions == test_labels_np)
accuracy = num_correct / float(test_size)
print("time required for predicting %d data point: %.2f seconds" % (test_size, run_time))
print("accuracy of model: %.1f%%" % (accuracy * 100))
predictions

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_seri

data split into 100 batches, of size 2.


The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_des

time required for predicting 176 data point: 1.26 seconds
accuracy of model: 100.0%


array([40457., 40846., 40846., 40784., 40487., 40620., 40267., 40650.,
       40015., 40457., 40484., 40360., 40784., 40650., 40015., 40340.,
       40457., 40595., 40230., 40210., 40210., 40487., 40487., 40360.,
       40457., 40410., 40287., 40340., 40623., 40680., 40846., 40288.,
       40680., 40483., 40595., 40599., 40067., 40770., 40435., 40015.,
       40466., 40487., 40267., 40814., 40410., 40846., 40360., 40015.,
       40360., 40457., 40210., 40210., 40650., 40585., 40287., 40287.,
       40210., 40680., 40457., 40067., 40487., 40286., 40287., 40554.,
       40846., 40266., 40650., 40487., 40680., 40410., 40784., 40360.,
       40210., 40680., 40410., 40680., 40435., 40457., 40680., 40487.,
       40210., 40846., 40457., 40585., 40487., 40210., 40585., 40457.,
       40210., 40680., 40595., 40585., 40554., 40770., 40340., 40680.,
       40680., 40487., 40487., 40015., 40680., 40286., 40585., 40554.,
       40340., 40340., 40210., 40680., 40680., 40435., 40015., 40585.,
      

Limpio los endpoints

In [42]:
def delete_endpoint(predictor):
    try:
        boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
        print('Deleted {}'.format(predictor.endpoint))
    except:
        print('Already deleted: {}'.format(predictor.endpoint))

In [1]:
#delete_endpoint(predictor)

NameError: name 'delete_endpoint' is not defined

In [55]:
batches

[array([[11.817397, 58.070121,  1.      ],
        [15.57309 , 58.395666,  1.      ]]),
 array([[15.612377, 58.396857,  1.      ],
        [17.651914, 59.852318,  1.      ]]),
 array([[22.141036, 65.600747,  1.      ],
        [20.968403, 64.753002,  1.      ]]),
 array([[16.445367, 61.35493 ,  1.      ],
        [13.750907, 57.361593,  1.      ]]),
 array([[12.67011, 58.23557,  1.     ],
        [11.88426, 57.94536,  1.     ]]),
 array([[20.624335, 67.154264,  1.      ],
        [14.099515, 57.902663,  1.      ]]),
 array([[18.064519, 60.06117 ,  1.      ],
        [14.531043, 56.73508 ,  1.      ]]),
 array([[12.68405 , 58.2395  ,  1.      ],
        [13.045035, 56.509235,  1.      ]]),
 array([[11.464418, 58.291643,  1.      ],
        [15.355387, 57.167062,  1.      ]]),
 array([[12.977963, 56.773311,  1.      ],
        [12.039695, 57.487158,  1.      ]]),
 array([[11.921073, 57.717211,  1.      ],
        [20.966141, 65.674821,  1.      ]]),
 array([[22.88525 , 65.818842,  1.    